# **Launch Sites Locations Analysis with Folium**


The launch success rate may depend on many factors such as payload mass, orbit type, and so on. It may also depend on the location and proximities of a launch site, i.e., the initial position of rocket trajectories. Finding an optimal location for building a launch site certainly involves many factors and hopefully we could discover some of the factors by analyzing the existing launch site locations.


n this lab, we will be performing more interactive visual analytics using `Folium`.


## Objectives


This lab contains the following tasks:

*   **TASK 1:** Mark all launch sites on a map
*   **TASK 2:** Mark the success/failed launches for each site on the map
*   **TASK 3:** Calculate the distances between a launch site to its proximities

After completed the above tasks, we find some geographical patterns about launch sites.


## Setup

Let's first import required Python packages for this notebook:


In [1]:
import folium
import pandas as pd

In [2]:
# Import folium MarkerCluster plugin
from folium.plugins import MarkerCluster
# Import folium MousePosition plugin
from folium.plugins import MousePosition
# Import folium DivIcon plugin
from folium.features import DivIcon

If you need to refresh your memory about folium, you may download and refer to this previous folium lab:


## Task 1: Mark all launch sites on a map


First, let's try to add each site's location on a map using site's latitude and longitude coordinates


The following dataset with the name `dataset_part_2.csv` is an augmented dataset with latitude and longitude added for each site.


In [4]:
# read the `dataset_part_2.csv`
path = "../../Data_Collecting&Wrangling/Wrangling/dataset_part_2.csv"
spacex_df=pd.read_csv(path)

Now, we can take a look at what are the coordinates for each site.


In [5]:
spacex_df.shape

(90, 18)

In [6]:
# Change relevant sub-columns to: `Launch Site`, `Lat(Latitude)`, `Long(Longitude)`, `class`
spacex_df = spacex_df[['LaunchSite', 'Latitude', 'Longitude', 'Class']]
spacex_df.columns = ['Launch Site', 'Lat', 'Long', 'class']
launch_sites_df = spacex_df.groupby(['Launch Site' ,'Lat', 'Long'], as_index=False).count()
# launch_sites_df = launch_sites_df[['Launch Site', 'Lat', 'Long']]
launch_sites_df

,Launch Site,Lat,Long,class
0,CCSFS SLC 40,28.561857,-80.577366,55
1,KSC LC 39A,28.608058,-80.603956,22
2,VAFB SLC 4E,34.632093,-120.610829,13


Above coordinates are just plain numbers that can not give you any intuitive insights about where are those launch sites. Let's visualize those locations by pinning them on a map.


We first need to create a folium `Map` object, with an initial center location to be NASA Johnson Space Center at Houston, Texas.


In [7]:
# Start location is NASA Johnson Space Center
nasa_coordinate = [29.559684888503615, -95.0830971930759]
site_map = folium.Map(location=nasa_coordinate, zoom_start=10)

We could use `folium.Circle` to add a highlighted circle area with a text label on a specific coordinate. For example,


In [8]:
# Create a blue circle at NASA Johnson Space Center's coordinate with a popup label showing its name
circle = folium.Circle(nasa_coordinate, radius=50, color='#d35400', fill=True).add_child(folium.Popup('NASA Johnson Space Center'))
# Create a blue circle at NASA Johnson Space Center's coordinate with a icon showing its name
marker = folium.map.Marker(
    nasa_coordinate,
    # Create an icon as a text label
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'NASA JSC',
        )
    )
site_map.add_child(circle)
site_map.add_child(marker)

generated map should be looks like this:

<center>
<img src="https://github.com/mrpintime/Falcon9_Landing/blob/master/Exploratory_DataAnalysis(EDA)/Interactive%20Visual%20Analytics%20with%20Folium/pictures/NASA.png?raw=true">
</center>

and you should find a small yellow circle near the city of Houston and you can zoom-in to see a larger circle.


Now, let's add a circle for each launch site in data frame `launch_sites`


In [9]:
# Initial the map
site_map = folium.Map(location=nasa_coordinate, zoom_start=5)
# For each launch site, add a Circle object based on its coordinate (Lat, Long) values. In addition, add Launch site name as a popup label
features = folium.map.FeatureGroup() 
# circle
for label,x,y in zip(launch_sites_df['Launch Site'],launch_sites_df.Lat, launch_sites_df.Long) :
    features.add_child(folium.Circle([x,y], radius=50, color='#d35400', fill=True).add_child(folium.Popup(
    label)))
    features.add_child(folium.Marker([x,y], icon=DivIcon(icon_size=(20,20),
    icon_anchor=(0,0), html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % label ))
    )
site_map.add_child(features)

The generated map with marked launch sites should look similar to the following:


<center>
    <img src="https://github.com/mrpintime/Falcon9_Landing/blob/master/Exploratory_DataAnalysis(EDA)/Interactive%20Visual%20Analytics%20with%20Folium/pictures/marker2.png?raw=true">
</center>

Now, you can explore the map by zoom-in/out the marked areas
, and we try to answer the following questions:

*   Are all launch sites in proximity to the Equator line?
*   Are all launch sites in very close proximity to the coast?


if we zoom in our map should be looks like this:

<center>
<img src="https://github.com/mrpintime/Falcon9_Landing/blob/master/Exploratory_DataAnalysis(EDA)/Interactive%20Visual%20Analytics%20with%20Folium/pictures/marker1.png?raw=true">
</center>

## Task 2: Mark the success/failed launches for each site on the map


Next, let's try to enhance the map by adding the launch outcomes for each site, and see which sites have high success rates.
Recall that data frame spacex_df has detailed launch records, and the `class` column indicates if this launch was successful or not


In [11]:
spacex_df.tail(10)

,Launch Site,Lat,Long,class
80,CCSFS SLC 40,28.561857,-80.577366,1
81,CCSFS SLC 40,28.561857,-80.577366,1
82,CCSFS SLC 40,28.561857,-80.577366,1
83,CCSFS SLC 40,28.561857,-80.577366,1
84,CCSFS SLC 40,28.561857,-80.577366,1
85,KSC LC 39A,28.608058,-80.603956,1
86,KSC LC 39A,28.608058,-80.603956,1
87,KSC LC 39A,28.608058,-80.603956,1
88,CCSFS SLC 40,28.561857,-80.577366,1
89,CCSFS SLC 40,28.561857,-80.577366,1


Next, let's create markers for all launch records.
If a launch was successful `(class=1)`, then we use a green marker and if a launch was failed, we use a red marker `(class=0)`


Note that a launch only happens in one of the four launch sites, which means many launch records will have the exact same coordinate. Marker clusters can be a good way to simplify a map containing many markers having the same coordinate.


Let's first create a `MarkerCluster` object


In [12]:
marker_cluster = MarkerCluster()

We create a new column in `launch_sites` dataframe called `marker_color` to store the marker colors based on the `class` value


In [13]:
marker_color = []
for i in spacex_df['class']:
    if i == 0:
        marker_color.append('red')
    else:
        marker_color.append('green')
spacex_df['marker_color'] = marker_color
spacex_df.head()

,Launch Site,Lat,Long,class,marker_color
0,CCSFS SLC 40,28.561857,-80.577366,0,red
1,CCSFS SLC 40,28.561857,-80.577366,0,red
2,CCSFS SLC 40,28.561857,-80.577366,0,red
3,VAFB SLC 4E,34.632093,-120.610829,0,red
4,CCSFS SLC 40,28.561857,-80.577366,0,red


For each launch result in `spacex_df` data frame, we add a `folium.Marker` to `marker_cluster`


In [14]:
# for index, record in spacex_df.iterrows():
#     print(index, record, sep='\n')
#     break

In [15]:
# Add marker_cluster to current site_map
# For each launch site, add a Circle object based on its coordinate (Lat, Long) values. In addition, add Launch site name as a popup label 
site_map.add_child(marker_cluster)
# for each row in spacex_df data frame
# create a Marker object with its coordinate
# and customize the Marker's icon property to indicate if this launch was successed or failed, 
# e.g., icon=folium.Icon(color='white', icon_color=row['marker_color']
for index, record in spacex_df.iterrows():
    marker = folium.Marker([record.Lat,record.Long], icon=folium.Icon(color='white', icon_color=record.marker_color ))
    marker_cluster.add_child(marker)

site_map

The map may look like the following screenshots:


<center>
    <img src="https://github.com/mrpintime/Falcon9_Landing/blob/master/Exploratory_DataAnalysis(EDA)/Interactive%20Visual%20Analytics%20with%20Folium/pictures/cluster1.png?raw=true">
</center>


<center>
    <img src="https://github.com/mrpintime/Falcon9_Landing/blob/master/Exploratory_DataAnalysis(EDA)/Interactive%20Visual%20Analytics%20with%20Folium/pictures/cluster1_1.png?raw=true">
</center>


<center>
    <img src="https://github.com/mrpintime/Falcon9_Landing/blob/master/Exploratory_DataAnalysis(EDA)/Interactive%20Visual%20Analytics%20with%20Folium/pictures/cluster2.png?raw=true">
</center>


From the color-labeled markers in marker clusters, we should be able to easily identify which launch sites have relatively high success rates.(KSC LC-39A)

## TASK 3: Calculate the distances between a launch site to its proximities


Next, we need to explore and analyze the proximities of launch sites.


Let's first add a `MousePosition` on the map to get coordinate for a mouse over a point on the map. As such, while you are exploring the map, you can easily find the coordinates of any points of interests (such as railway)


In [17]:
# Add Mouse Position to get the coordinate (Lat, Long) for a mouse over on the map
formatter = "function(num) {return L.Util.formatNum(num, 5);};"
mouse_position = MousePosition(
    position='topright',
    separator=' Long: ',
    empty_string='NaN',
    lng_first=False,
    num_digits=20,
    prefix='Lat:',
    lat_formatter=formatter,
    lng_formatter=formatter,
)

# site_map.add_child(mouse_position)
site_map.add_child(mouse_position)
site_map

You can see mouse position show us latitude and longitude at top right of map, looks like following screenshot:

<center>
    <img src="https://github.com/mrpintime/Falcon9_Landing/blob/master/Exploratory_DataAnalysis(EDA)/Interactive%20Visual%20Analytics%20with%20Folium/pictures/mouseposition.png?raw=true">
</center>


In [18]:
# for finding children you have already added to map
site_map._children

OrderedDict([('openstreetmap',
              <folium.raster_layers.TileLayer at 0x7fde34c66890>),
             ('feature_group_b4dc0c82da48d0c56194e7d3abdd6b38',
              <folium.map.FeatureGroup at 0x7fde754f69b0>),
             ('marker_cluster_0b2b71c3698d961088896d7920f88bec',
              <folium.plugins.marker_cluster.MarkerCluster at 0x7fde34c675b0>),
             ('mouse_position_1b5d48fc0f507af68b706c0ef5d1a279',
              <folium.plugins.mouse_position.MousePosition at 0x7fde34c00fa0>)])

Create function for calculate distance of two points on map in KM


In [19]:
from math import sin, cos, sqrt, atan2, radians
# create function for calculate distance of two points in KM
def calculate_distance(lat1, lon1, lat2, lon2):
    # approximate radius of earth in km
    R = 6373.0

    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return distance

Mark all closest points to our launch sites and calculate distance of them from launch sites

In [20]:
launch_sites_df

,Launch Site,Lat,Long,class
0,CCSFS SLC 40,28.561857,-80.577366,55
1,KSC LC 39A,28.608058,-80.603956,22
2,VAFB SLC 4E,34.632093,-120.610829,13


In [21]:
launch_sites_df.loc[1, 'Lat']

28.6080585

In [22]:
# Launch sites 
#coordinates format [lat, lon]

CCSFS_SLC = {'coor':[launch_sites_df.loc[0, 'Lat'], launch_sites_df.loc[0, 'Long']]}
CCSFS_SLC_property = {'coastln':[28.56339, -80.56803], 'railway':[28.5726, -80.5852], 'highway':[28.5632, -80.5707], 'city':[28.5604, -80.814]}

# CCAFS_LC = {'coor':[launch_sites_df.loc[1, 'Lat'], launch_sites_df.loc[1, 'Long']]}
# CCAFS_LC_property = {'coastln':[28.56208, -80.56784], 'railway':[28.5721, -80.5853], 'highway':[28.5623, -80.5706], 'city':[28.4022, -80.595]}

KSC_LC = {'coor':[launch_sites_df.loc[1, 'Lat'], launch_sites_df.loc[1, 'Long']]}
KSC_LC_property = {'coastln':[28.61151, -80.5974], 'railway':[28.61186, -80.59807], 'highway':[28.6116, -80.59916], 'city':[28.591, -80.814]}

VAFB_SLC = {'coor':[launch_sites_df.loc[2, 'Lat'], launch_sites_df.loc[2, 'Long']]}
VAFB_SLC_property = {'coastln':[34.63415, -120.62561], 'railway':[34.633, -120.625], 'highway':[34.63347, -120.62362], 'city':[34.638, -120.459]}

In [23]:
launch_list_coor = [CCSFS_SLC, KSC_LC, VAFB_SLC]
launch_list_prop = [CCSFS_SLC_property, KSC_LC_property, VAFB_SLC_property]

In [24]:
# We find coordinate of the closet coastline in map using MousePosition
# Launch_site coordinates is for "CCAFS LC-40"
# launch_site_lat = 28.562302
# launch_site_lon = -80.577356
# coastline_lat = 28.564
# coastline_lon = -80.568
# distance_coastline = calculate_distance(launch_site_lat, launch_site_lon, coastline_lat, coastline_lon)
# distance_coastline

Draw a `PolyLine` between a launch site to the selected closest highway, coastline, railroad,etc.. points


In [25]:
# Create and add a folium.Marker on your selected closest selected location point on the map
# Display the distance between points and launch site using the icon property 
# blue for coastline, red for railway, green for highway, purple for city 
colors = ['blue', 'red', 'green', 'purple']
for c,p in zip(launch_list_coor, launch_list_prop):
    point_loc = [c['coor'][0], c['coor'][1]]
    # draw marker and line between selected launchSite and closest locations
    for k,color in zip(p, colors):
        distance = calculate_distance(point_loc[0], point_loc[1], p[k][0], p[k][1])
        coordinate = [p[k][0], p[k][1]]
        distance_marker = folium.Marker(
           coordinate,
           icon=DivIcon(
               icon_size=(20,20),
               icon_anchor=(0,0),
               html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % "{:10.2f} KM".format(distance),
               ))
        distance_marker.add_to(site_map)
        # print(p[k])
        
        # Create a line with `folium.PolyLine` object using the coastline,highway,railroad coordinates and launch site coordinate
        launch_site_lat, launch_site_lon, coastline_lat, coastline_lon = point_loc[0], point_loc[1], p[k][0], p[k][1]
        coordinates_line = [[launch_site_lat, launch_site_lon], [coastline_lat, coastline_lon]]
        line=folium.PolyLine(locations=coordinates_line, weight=1, color=color)
        site_map.add_child(line)


In [26]:
site_map

Our updated map with distance lines and should looks like the following screenshots if we zoom in:


<center>
    <img src="https://github.com/mrpintime/Falcon9_Landing/blob/master/Exploratory_DataAnalysis(EDA)/Interactive%20Visual%20Analytics%20with%20Folium/pictures/clusterlines1.png?raw=true">
</center>


<center>
    <img src="https://github.com/mrpintime/Falcon9_Landing/blob/master/Exploratory_DataAnalysis(EDA)/Interactive%20Visual%20Analytics%20with%20Folium/pictures/clusterlines2.png?raw=true">
</center>

A railway map symbol may look like this:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/railway.png">
</center>


A highway map symbol may look like this:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/highway.png">
</center>


A city map symbol may look like this:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/city.png">
</center>


After we plot distance lines to the proximities, we can answer the following questions easily:

*   Are launch sites in close proximity to railways?
*   Are launch sites in close proximity to highways?
*   Are launch sites in close proximity to coastline?
*   Do launch sites keep certain distance away from cities?


## Author


<center>Moein (mrpintime)</center>